In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import numpy as np
from imblearn import under_sampling
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score

from imblearn.datasets import fetch_datasets
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier
from imblearn.ensemble import RUSBoostClassifier

In [2]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
train_data=pd.read_csv('train.csv')
test_data=pd.read_csv('test.csv')
for c in test_data.columns:
    if(test_data[c].dtype==object and c!='Complaint-ID' and c!='Date-received' and c!='Date-sent-to-company'):
        le=LabelEncoder()
        le.fit(train_data[c].astype(str))
        train_data[c]=le.transform(train_data[c].astype(str))
        test_data[c] = test_data[c].map(lambda s: '<unknown>' if s not in le.classes_ else s)
        le.classes_ = np.append(le.classes_, '<unknown>')
        test_data[c]=le.transform(test_data[c].astype(str))
       

In [4]:
train_data['Date-received']=pd.to_datetime(train_data['Date-received'])

train_data['Year']=train_data['Date-received'].dt.year

train_data['Month']=train_data['Date-received'].dt.month

train_data['day']=train_data['Date-received'].dt.day

In [5]:
test_data['Date-received']=pd.to_datetime(test_data['Date-received'])

test_data['Year']=test_data['Date-received'].dt.year

test_data['Month']=test_data['Date-received'].dt.month

test_data['day']=test_data['Date-received'].dt.day


In [6]:
train_data['Complaint-Status']=train_data['Complaint-Status'].astype('category')
train_data['Complaint-Status']=train_data['Complaint-Status'].cat.codes

In [7]:
def fun(x):
    if(x==3):
        return 1
    elif x==2:
        return 0
    else:
        return 2
        
train_data['status']=train_data['Complaint-Status'].apply(fun)

In [8]:
target=train_data['status']
t_data,d_data,t_label,d_label=train_test_split(train_data,target,test_size=0.10)

In [9]:
#train_data=t_data

# seprate for monetary and non monetary 

In [19]:
train_data['Complaint-Status'].value_counts()

1    34300
3     5018
2     2818
0      809
4      321
Name: Complaint-Status, dtype: int64

In [20]:
required_columns=[u'Transaction-Type', u'Complaint-reason', u'Company-response',
         u'Consumer-disputes', u'Year', u'Month', u'day']

# Classify as monetary or non monetary or others 

In [12]:
model=xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05,scale_pos_weight=0.8,min_child_weight=1)
model.fit(t_data[required_columns],t_label)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=0.8, seed=None,
       silent=True, subsample=1)

In [13]:
pred=model.predict(d_data[required_columns])

In [14]:
pred=pd.DataFrame(pred)

In [15]:
pred[0].value_counts()

2    4281
0      32
1      14
Name: 0, dtype: int64

In [16]:
from sklearn.metrics import f1_score
score1=f1_score(d_label,pred, average='weighted')

In [17]:
score1

0.7505705604288863

# Only 2 and 3 class

In [9]:
data3=train_data[train_data['Complaint-Status']==3]

In [12]:
data2=train_data[train_data['Complaint-Status']==2]

In [13]:
data23=pd.concat([data2,data3],0)

In [15]:
data23.shape

(7836, 13)

In [16]:
target=data23['Complaint-Status']

In [17]:
t_data,d_data,t_label,d_label=train_test_split(data23,target,test_size=0.10)

In [18]:
t_data.shape

(7052, 13)

In [21]:
model=xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05,scale_pos_weight=0.8,min_child_weight=1)
model.fit(t_data[required_columns],t_label)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=0.8, seed=None,
       silent=True, subsample=1)

In [24]:
pred=model.predict(d_data[required_columns])

In [25]:
pred=pd.DataFrame(pred)

In [26]:
pred[0].value_counts()

3    435
2    349
Name: 0, dtype: int64

# There must be Seprate classifier for class 2 and 3